# Emails Spam Detection

This notebook uses a Naive Bayes model in order to train a spam classifier.

# Imports

In [7]:
import os
import io
import numpy
from pandas import DataFrame
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB

# Get Data

In [8]:
def readFiles(path):
    for root, dirnames, filenames in os.walk(path):
        for filename in filenames:
            path = os.path.join(root, filename)

            inBody = False
            lines = []
            f = io.open(path, 'r', encoding='latin1')
            for line in f:
                if inBody:
                    lines.append(line)
                elif line == '\n':
                    inBody = True
            f.close()
            message = '\n'.join(lines)
            yield path, message


def dataFrameFromDirectory(path, classification):
    rows = []
    index = []
    for filename, message in readFiles(path):
        rows.append({'message': message, 'class': classification})
        index.append(filename)

    return DataFrame(rows, index=index)

In [9]:
data = DataFrame({'message': [], 'class': []})

data = data.append(dataFrameFromDirectory('datasets/emails/spam', 'spam'))
data = data.append(dataFrameFromDirectory('datasets/emails/ham', 'ham'))

In [4]:
data.head()

,message,class
datasets/emails/spam\00001.7848dde101aa985090474a91ec93fcf0,"<!DOCTYPE HTML PUBLIC ""-//W3C//DTD HTML 4.0 Tr...",spam
datasets/emails/spam\00002.d94f1b97e48ed3b553b3508d116e6a09,1) Fight The Risk of Cancer!\n\nhttp://www.adc...,spam
datasets/emails/spam\00003.2ee33bc6eacdb11f38d052c44819ba6c,1) Fight The Risk of Cancer!\n\nhttp://www.adc...,spam
datasets/emails/spam\00004.eac8de8d759b7e74154f142194282724,##############################################...,spam
datasets/emails/spam\00005.57696a39d7d84318ce497886896bf90d,I thought you might like these:\n\n1) Slim Dow...,spam


# Train

In [15]:
vectorizer = CountVectorizer()
counts = vectorizer.fit_transform(data['message'].values)

classifier = MultinomialNB()
targets = data['class'].values

classifier.fit(counts, targets)

MultinomialNB()

# Evaluate

In [13]:
examples = ['Free Viagra now!!!', "Hi Bob, how about a game of golf tomorrow?"] #...
example_counts = vectorizer.transform(examples)
predictions = classifier.predict(example_counts)

predictions

array(['spam', 'ham'], dtype='<U4')